In [2]:
import os
import numpy as np
import scipy.misc
from stylize import stylize
import math
from argparse import ArgumentParser


import vgg
import tensorflow as tf
import numpy as np
from sys import stderr

In [3]:
# default arguments
CONTENT_WEIGHT = 5e0
STYLE_WEIGHT = 1e2
TV_WEIGHT = 1e2
LEARNING_RATE = 1e1
STYLE_SCALE = 1.0
ITERATIONS = 1000
VGG_PATH = 'imagenet-vgg-verydeep-19.mat'

CONTENT_LAYER = 'relu4_2'
STYLE_LAYERS = ('relu1_1', 'relu2_1', 'relu3_1', 'relu4_1', 'relu5_1')

In [4]:
def imread(path):
    return scipy.misc.imread(path).astype(np.float)

In [5]:
class Args(dict):
    pass

options = Args()
options.content = 'examples/2-content.jpg'
options.styles = ['examples/2-style1.jpg']
options.output = 'examples/myoutput.jpg'
options.iterations = 1
options.network = VGG_PATH

In [6]:
options.content

'examples/2-content.jpg'

In [7]:
content_image = imread(options.content)
style_images = [imread(style) for style in options.styles]
target_shape = content_image.shape
for i in range(len(style_images)):
    style_scale = STYLE_SCALE
    #resize style image according to width faction
    style_images[i] = scipy.misc.imresize(style_images[i], style_scale *
            target_shape[1] / style_images[i].shape[1])

In [8]:
content = content_image
styles = style_images
iterations = options.iterations
network = options.network

In [9]:
#initialize shape and features
shape = (1,) + content.shape
style_shapes = [(1,) + style.shape for style in styles]
content_features = {}
style_features = [{} for _ in styles]

In [10]:
g = tf.Graph()
# compute content features in feedforward mode
with g.as_default(), g.device('/cpu:0'), tf.Session() as sess:
    image = tf.placeholder('float', shape=shape)
    net, mean_pixel = vgg.net(network, image)
    content_pre = np.array([vgg.preprocess(content, mean_pixel)])
    content_features[CONTENT_LAYER] = net[CONTENT_LAYER].eval(
            feed_dict={image: content_pre})

In [13]:
# compute style features in feedforward mode
for i in range(len(styles)):
    g = tf.Graph()
    with g.as_default(), g.device('/cpu:0'), tf.Session() as sess:
        image = tf.placeholder('float', shape=style_shapes[i])
        net, _ = vgg.net(network, image)
        style_pre = np.array([vgg.preprocess(styles[i], mean_pixel)])
        for layer in STYLE_LAYERS:
            features = net[layer].eval(feed_dict={image: style_pre})
            features = np.reshape(features, (-1, features.shape[3]))
            gram = np.matmul(features.T, features) / features.size
            style_features[i][layer] = gram

<img src="features.png">
To create a graph with certain content and style, we want to first acquire the features of these contents and styles as a standard. Which means, we want our image to have both features included in content image and features in style images. Unsurprisingly, neurons in neural network are a great way to show these features according to its intuition. Thus, we utilize the pre-trained vgg network, input content image and style images and compute output of each layer in the network. Then, we choose some of the layers as features of the content and style.

After having those features in hand, how can we create the stylized image? If we have an image that has has all the features in content and style images, then this image is exatly what we want. So let's first solve this problem - given an image, how can we measure the difference between this image and ideal image that has all the features? One thing we could do is to input our image into the neural network, compute output in each layer, pick certain layer as its features and compare its feature with corresponding content and style feature. The comparison could achieved using the loss function. 

As we form the loss function, our goal becomes to find an image that minimizes this loss function. This goal can be easily achieved using the backpropagation method using in neural network training process - We input a white noise image and use gradient descent to gradually change each pixel in the image to minimize the loss function.  
